<a href="https://colab.research.google.com/github/namhihict3129-collab/DATA-COLLECTION/blob/main/DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install geemap

In [22]:
import ee
import geemap

In [27]:
ee.Authenticate()
ee.Initialize(project='deft-virtue-475711-b0')

In [28]:
Map = geemap.Map()
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [29]:


# Định nghĩa vùng TP.HCM
aoi = ee.Geometry.Point([106.660172, 10.762622])  # Hoặc bao QUẬN 10, TP.HCM...

# Truy xuất bộ dữ liệu Sentinel-5P NO2 (2025-10)
no2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
    .filterDate('2025-10-01', '2025-10-20') \
    .filterBounds(aoi) \
    .select('tropospheric_NO2_column_number_density')

# Lấy giá trị trung bình NO2 cho khu vực
no2_mean = no2.mean().reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=1000
)
print('NO2 mean:', no2_mean.getInfo())

# Truy xuất ERA5 nhiệt độ, độ ẩm, gió
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate('2025-10-01', '2025-10-20') \
    .filterBounds(aoi)

temp = era5.select('mean_2m_air_temperature').mean().reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=1000
)
humidity = era5.select('mean_surface_relative_humidity').mean().reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=1000
)
wind_u = era5.select('u_component_of_wind_10m').mean().reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=1000
)
wind_v = era5.select('v_component_of_wind_10m').mean().reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=1000
)

print('Temperature (K):', temp.getInfo())
print('Humidity (%):', humidity.getInfo())
print('Wind U (m/s):', wind_u.getInfo())
print('Wind V (m/s):', wind_v.getInfo())


NO2 mean: {'tropospheric_NO2_column_number_density': 6.292946800433844e-05}
Temperature (K): {}
Humidity (%): {}
Wind U (m/s): {}
Wind V (m/s): {}


In [30]:
jan_2023_climate = (
    ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
    .filterDate('2023-01', '2023-02')
    .first()
)
jan_2023_climate

In [31]:
m = geemap.Map(center=[30, 0], zoom=2)

vis_params = {
    'bands': ['temperature_2m'],
    'min': 229,
    'max': 304,
    'palette': 'inferno',
}
m.add_layer(jan_2023_climate, vis_params, 'Temperature (K)')
m

Map(center=[30, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…